<a href="https://colab.research.google.com/github/yuukimotai/GA4_QABot/blob/main/TouhokuUniv_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!touch bert_config.json

[参考記事](https://qiita.com/Mizuiro__sakura/items/326ee3e4ee58cf55e92c)

In [ ]:
!touch tokenizer_config.json

In [36]:
access_token = ""

In [ ]:
import gdown

# ファイルIDを指定
file_id = "1-4dLB4T9h0XP-rpvYnbXvxBBnXW1vegi"
url = f"https://drive.google.com/uc?id={file_id}"

# ファイルの保存先を指定
output = "./pytorch_model.bin"

# ファイルをダウンロード
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-4dLB4T9h0XP-rpvYnbXvxBBnXW1vegi
From (redirected): https://drive.google.com/uc?id=1-4dLB4T9h0XP-rpvYnbXvxBBnXW1vegi&confirm=t&uuid=2b253e67-8b89-4223-b22e-9736248cf8b9
To: /content/pytorch_model.bin
100%|██████████| 443M/443M [00:07<00:00, 55.8MB/s]


'./pytorch_model.bin'

In [ ]:
import gdown

# ファイルIDを指定
file_id = "1DubBO3fagYccVuKwrIVa4zGvsCzW7FCr"
url = f"https://drive.google.com/uc?id={file_id}"

# ファイルの保存先を指定
output = "./ckpt.pth"

# ファイルをダウンロード
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1DubBO3fagYccVuKwrIVa4zGvsCzW7FCr
From (redirected): https://drive.google.com/uc?id=1DubBO3fagYccVuKwrIVa4zGvsCzW7FCr&confirm=t&uuid=26eef817-a4e3-4295-867e-86b22ab18e7b
To: /content/ckpt.pth
100%|██████████| 1.79G/1.79G [00:28<00:00, 61.9MB/s]


'./ckpt.pth'

In [ ]:
pytorch_model_token = ""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese", token=access_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
with open("kanji_kenji_token.txt", "r") as f:
  # Read the lines of the file into a list.
  kanji_kenji_token = f.readlines()

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese", token=access_token)
tokenized_data = [tokenizer(text) for text in kanji_kenji_token]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Token indices sequence length is longer than the specified maximum sequence length for this model (118640 > 512). Running this sequence through the model will result in indexing errors


In [38]:
!pip install fugashi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 5.2 MB/s eta 0:00:00


In [40]:
!pip install ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=5e64d43e6ce21ef57e777c21f9357a7867e6c1b31c93ff37da35857b5d66018b
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


In [41]:
from transformers import AutoConfig, AutoTokenizer, BertForQuestionAnswering
import torch
config = AutoConfig.from_pretrained("config.json")
tokenizer_config = AutoConfig.from_pretrained("tokenizer.json")
model = BertForQuestionAnswering.from_pretrained('pytorch_model.bin',config=config) # 学習済みモデルの選択
torch.save(model.state_dict(),'model.pth') # モデルを一度保存してキーに関するエラーが起こらないようにする
model.load_state_dict(torch.load('model.pth',map_location=torch.device('cpu'))) # 学習済みモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese', token=access_token) # tokenizerの読み込み

In [42]:
def predict(question,context):
    input_ids=tokenizer.encode(question,context) # tokenizerで形態素解析しつつコードに変換する
    output= model(torch.tensor([input_ids])) # 学習済みモデルを用いて解析
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids) # コード化した文章を復号化する
    prediction = ''.join(all_tokens[torch.argmax(output.start_logits) : torch.argmax(output.end_logits)+1]) # 答えに該当する部分を抜き取る
    prediction = prediction.replace("#", "") # 余分な文字を削除する
    prediction = prediction.replace(" ","")
    prediction = prediction.replace("[SEP]","")
    return prediction

In [45]:
context = "チョコミントの材料はチョコとミント"
question = "チョコミントの材料って何？"
return_message = predict(question,context)
print(return_message)

チョコとミント
